In [45]:
import multiprocessing, torch, re
import os.path as osp
import datasets

from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig,\
    DataCollatorForLanguageModeling, Trainer, TrainingArguments, BertForMaskedLM, FillMaskPipeline
from tokenization_kobert import KoBertTokenizer
from datasets import load_dataset
from torch.utils.data import Dataset
from tqdm.auto import tqdm
from time import strftime, time, localtime
from os import listdir
from functools import partial

print(torch.__version__)

2.0.0+cu117


In [2]:
# model_path = "skt/kogpt2-base-v2"
# tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path, 
#                                                     bos_token='</s>', eos_token='</s>', unk_token='<unk>',
#                                                     pad_token='<pad>', mask_token='<mask>', max_length=512)
# model = GPT2LMHeadModel.from_pretrained(model_path)
# model = BertForMaskedLM.from_pretrained("skt/kobert-base-v1")
# model = AutoModel.from_pretrained('monologg/distilkobert')
model = BertForMaskedLM.from_pretrained("monologg/kobert-lm")
tokenizer = KoBertTokenizer.from_pretrained('monologg/distilkobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [3]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [4]:
tokenizer

KoBertTokenizer(name_or_path='monologg/distilkobert', vocab_size=8002, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [5]:
sent = "안녕하세요.[MASK][MASK][MASK][MASK][MASK][MASK]"

# collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
encoded = tokenizer(sent, return_tensors="pt")
# target = tokenizer("안녕하세요", return_tensors='pt')

out = model(**encoded)
pipe = FillMaskPipeline(model=model, tokenizer=tokenizer)
# while True:
#     t = model(**encoded)
#     logits:torch.Tensor = t.logits
#     predicted_token = logits[:,-1,:].argmax(dim=1)
#     print(tokenizer.decode(predicted_token))

# print(tokenizer.decode(logits[:,-1,:].argmax(dim=1)))

In [6]:
pipe("안녕하[MASK][MASK]")

[[{'score': 0.6325711607933044,
   'token': 6749,
   'token_str': '십',
   'sequence': '[CLS] 안녕하십[MASK][SEP]'},
  {'score': 0.10633417218923569,
   'token': 6586,
   'token_str': '세요',
   'sequence': '[CLS] 안녕하세요[MASK][SEP]'},
  {'score': 0.05746186524629593,
   'token': 6579,
   'token_str': '세',
   'sequence': '[CLS] 안녕하세[MASK][SEP]'},
  {'score': 0.0350675955414772,
   'token': 7275,
   'token_str': '죠',
   'sequence': '[CLS] 안녕하죠[MASK][SEP]'},
  {'score': 0.022893358021974564,
   'token': 6601,
   'token_str': '셔',
   'sequence': '[CLS] 안녕하셔[MASK][SEP]'}],
 [{'score': 0.7970646619796753,
   'token': 54,
   'token_str': '.',
   'sequence': '[CLS] 안녕하[MASK].[SEP]'},
  {'score': 0.12322165817022324,
   'token': 258,
   'token_str': '?',
   'sequence': '[CLS] 안녕하[MASK]?[SEP]'},
  {'score': 0.03613218292593956,
   'token': 5771,
   'token_str': '니까',
   'sequence': '[CLS] 안녕하[MASK]니까[SEP]'},
  {'score': 0.012106643058359623,
   'token': 6999,
   'token_str': '요',
   'sequence': '[CLS] 안

In [7]:
mask_idxs = encoded['input_ids'] == tokenizer.mask_token_id
mask_idxs

tensor([[False, False, False, False,  True,  True, False]])

In [8]:
out.logits.shape

torch.Size([1, 7, 8002])

In [9]:
masks = out.logits[mask_idxs].argmax(dim=-1)
masks

tensor([6749,   54])

In [10]:
tokenizer.decode(masks)

'십.'

In [11]:
predicted_token = out.logits[0,-1].argmax(-1)
tokenizer.decode(predicted_token)

'요'

In [12]:
# dataset = load_dataset("HuggingFaceH4/ultrachat_200k")
# dataset = load_dataset("wikipedia", "20220301.en")
# dataset = dataset.remove_columns([col for col in dataset.column_names if col != 'text'])
# dataset = load_dataset("bookcorpus", split="train", streaming=True).with_format('torch')
# dataset = load_dataset("bookcorpus", split="train")
dataset = load_dataset("text", data_dir="data")
# len(dataset)

Resolving data files:   0%|          | 0/98652 [00:00<?, ?it/s]

Using custom data configuration default-6cd95896b7a84724
Found cached dataset text (C:/Users/sizzf/.cache/huggingface/datasets/text/default-6cd95896b7a84724/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad)


  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1657394
    })
})

In [14]:
dataset = dataset["train"].train_test_split(test_size=0.3)

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1160175
    })
    test: Dataset({
        features: ['text'],
        num_rows: 497219
    })
})

In [16]:
s = dataset['train'][0]['text']
s 

'2 : 맞아'

2 : 


'맞아'

In [18]:
def get_time_dir(): return f"tokenizer_{strftime('%m-%d-%H-%M', localtime(time()))}"

In [19]:
# def batch_iterator(batch_size=10000):
#     for i in tqdm(range(0, len(dataset['train']), batch_size)):
#         yield dataset['train'][i:i+batch_size]['text']
# if [_dir for _dir in listdir() if "tokenizer" in _dir] != []:
#     latest_tokenizer_path = sorted([_dir for _dir in listdir() if "tokenizer" in _dir])[-1]
#     tokenizer = AutoTokenizer.from_pretrained(latest_tokenizer_path)
# else:
#     # tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2").train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32_000)
#     tokenizer = AutoTokenizer.from_pretrained("skplanet/dialog-koelectra-small-discriminator").train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32_000)
#     tokenizer.save_pretrained(get_time_dir())

# tokenizer.pad_token = '<pad>'
# tokenizer.eos_token = '</s>'
# tokenizer.bos_token = '</s>'
# tokenizer.unk_token = '<unk>'
# tokenizer.mask_token = '<mask>'
# tokenizer.model_max_length = 32

In [20]:
num_proc = multiprocessing.cpu_count()
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

The max length for the tokenizer is: 512


In [ ]:
tokenizer.convert_tokens_to_ids

In [54]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("안녕하세요"))

[3135, 5724, 7814]

In [50]:
def group_texts(examples:datasets.arrow_dataset.Batch, tokenizer=tokenizer):
    print(type(examples))
    print(examples['text'])
    
    for i, s in enumerate(examples['text']):
        for match in reversed([*re.finditer("^[0-9]* : ", s)]):
            s = s[:match.start()] + s[match.end():]
        examples['text'][i] =s
    
    label:str = ""
    tokenized_label = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(label))
    
    tokenized_inputs = tokenizer(
        examples['text'], return_special_tokens_mask=True, truncation=True, max_length=tokenizer.model_max_length, padding=True
    )
    return tokenized_inputs|{"label":tokenized_label}

In [47]:
samples = load_dataset("text", data_files=["data/BAND_11_05.txt"])

Using custom data configuration default-350f673cbd97f766
Found cached dataset text (C:/Users/sizzf/.cache/huggingface/datasets/text/default-350f673cbd97f766/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad)


  0%|          | 0/1 [00:00<?, ?it/s]

In [51]:
# tokenized_dataset = dataset.map(partial(group_texts, tokenizer=tokenizer), batched=True, remove_columns=['text'], num_proc=num_proc)
# tokenized_dataset = dataset.map(partial(group_texts, tokenizer=tokenizer), batched=True, remove_columns=['text'])
# tokenized_dataset = dataset.map(partial(group_texts, tokenizer=tokenizer), batched=True, remove_columns=['text'])
mapped = samples.map(partial(group_texts, tokenizer=tokenizer), batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

<class 'datasets.arrow_dataset.Batch'>
['1 : 야 니 동생 이번에 나온다고 하더니!', '2 : 맞아 근데 코로나 때문에 잘렸어!', '1 : 역시 코로나 갑 오브 갑이네 키키', '2 : 그러게 하지만 금방 전역한대', '1 : 헐 벌써 시간이 그렇게 됐나?', '2 : 내 말이 아주 두려워 죽겠어 키키', '1 : 근데 원래 걔 휴가 날이 언제였지?', '2 : 음 잠시만 8월 중순 쯤이라 하던데?', '1 : 그래서 사람들한테 7월 말에 전화 돌렸구나?', '2 : 어 맞아 키키 아주 오지랖이 넓어', '1 : 그러니까 거기서 약술도 팔고 말이야 키키', '2 : 아빠 닮아서 장사꾼 다됐어 이런!', '1 : 그래도 잘 지내고 있네!', '2 : 맞아 살도 뒤룩뒤룩 쪘어 ㅠㅠ 으이구!', '1 : 야 걔는 운동하러 가서 왜 살이 쪘냐 ㅜㅜ', '2 : 그래도 운동 열심히 한대', '2 : 곧 나오니깐 같이 한번 보자!']
1 : 
야 니 동생 이번에 나온다고 하더니!
2 : 
맞아 근데 코로나 때문에 잘렸어!
1 : 
역시 코로나 갑 오브 갑이네 키키
2 : 
그러게 하지만 금방 전역한대
1 : 
헐 벌써 시간이 그렇게 됐나?
2 : 
내 말이 아주 두려워 죽겠어 키키
1 : 
근데 원래 걔 휴가 날이 언제였지?
2 : 
음 잠시만 8월 중순 쯤이라 하던데?
1 : 
그래서 사람들한테 7월 말에 전화 돌렸구나?
2 : 
어 맞아 키키 아주 오지랖이 넓어
1 : 
그러니까 거기서 약술도 팔고 말이야 키키
2 : 
아빠 닮아서 장사꾼 다됐어 이런!
1 : 
그래도 잘 지내고 있네!
2 : 
맞아 살도 뒤룩뒤룩 쪘어 ㅠㅠ 으이구!
1 : 
야 걔는 운동하러 가서 왜 살이 쪘냐 ㅜㅜ
2 : 
그래도 운동 열심히 한대
2 : 
곧 나오니깐 같이 한번 보자!


In [49]:
samples, mapped

(DatasetDict({
     train: Dataset({
         features: ['text'],
         num_rows: 17
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['text', 'input_ids', 'token_type_ids', 'special_tokens_mask', 'attention_mask'],
         num_rows: 17
     })
 }))

In [23]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1160175
    })
    test: Dataset({
        features: ['text'],
        num_rows: 497219
    })
})

In [ ]:
tokenized_dataset

In [ ]:
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size = len(tokenizer),
    n_ctx = tokenizer.model_max_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
args = TrainingArguments(
    output_dir=f"AutocompleteLM-{get_time_dir()}",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    # num_train_epochs=1,
    weight_decay=.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    max_steps=74004228 * (num_train_epochs:=1),
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
)

In [ ]:
model

In [ ]:
trainer.train()

In [ ]:
s = "안녕하세요, "
tokens = tokenizer(s, return_tensors="pt")
output = model.generate(**tokens)

In [ ]:
tokens['input_ids'].shape, output.shape

In [ ]:
tokenizer.decode(output[0])